In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np
import ast
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ✅ Mount Google Drive (uncomment in Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# ✅ Load dataset from drive
folder_path = "/content/drive/MyDrive/ML_2"
movies_df = pd.read_csv(folder_path + "/tmdb_5000_movies.csv")
credits_df = pd.read_csv(folder_path + "/tmdb_5000_credits.csv")

# ✅ Merge datasets
movies_df = movies_df.merge(credits_df, on='title')

# ✅ Keep important columns ONLY
movies_df = movies_df[['title', 'genres', 'overview', 'release_date']]

# ✅ Convert genres JSON → list
def convert(obj):
    try:
        obj = ast.literal_eval(obj)
        return [i['name'] for i in obj]
    except:
        return []

movies_df['genres'] = movies_df['genres'].apply(convert)

# ✅ Fill missing overview safely
movies_df['overview'] = movies_df['overview'].fillna('')

# ✅ Extract year
movies_df['year'] = movies_df['release_date'].astype(str).str[:4]
movies_df['year'] = movies_df['year'].fillna("Unknown")

# ✅ Fake rating generator (because TMDB_5000 doesn't have MPAA ratings)
# Map movie year to rough age category (just to support UI filters)
def guess_rating_by_year(year):
    year = str(year)
    if year == "Unknown":
        return "PG-13"
    y = int(year) if year.isdigit() else 2000
    if y < 1990:
        return "PG"
    if y < 2010:
        return "PG-13"
    return "R"

movies_df['rating'] = movies_df['year'].apply(guess_rating_by_year)

# ✅ Convert rating → Age group
def assign_age_group(rating):
    rating = str(rating)
    if rating in ["G", "PG"]:
        return "Kids (0-12)"
    if rating == "PG-13":
        return "Teens (13-17)"
    return "Adults (18+)"

movies_df['age_group'] = movies_df['rating'].apply(assign_age_group)

# ✅ Prepare combined text for TF-IDF
movies_df['tags'] = movies_df['overview'] + " " + movies_df['genres'].apply(lambda x: " ".join(x))

# ✅ TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = tfidf.fit_transform(movies_df['tags']).toarray()

# ✅ Calculate similarity
similarity = cosine_similarity(vectors)

# ✅ Save movie data
movie_dict = movies_df.to_dict()

pickle.dump(movie_dict, open('movie_dict.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))

print("✅ Model training completed successfully!")
print("📁 Files saved: movie_dict.pkl & similarity.pkl")
print(f"🎬 Total Movies Trained: {len(movies_df)}")
